In [1]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
from pytorch_lightning import LightningDataModule
from torchaudio.datasets.librispeech import LIBRISPEECH

from examples.self_supervised_learning.data_modules._utils import BucketizeBatchSampler, CommonTransform
from examples.self_supervised_learning.data_modules._wav2vec2_datamodule import *
from torchaudio.prototype.models._conformer_wav2vec2 import * 
import torch.nn.functional as F
import pickle
from pathlib import Path
from tqdm import tqdm


librispeech_cls = LIBRISPEECH
dataset = librispeech_cls(root="..", url="train-clean-100", download=False) # setup root properly, download=True only if you need to download the dataset 
dataset = TransformDataset(dataset, self.train_transform)
pp.pprint(dataset[0][0].shape)


torch.Size([1, 225360])


In [2]:
len_list_path = Path('len_list.obj')
len_list = []
if len_list_path.is_file():
    with open('len_list.obj', 'rb') as fp:
        len_list = pickle.load(fp)
else:
    len_list = [d[0].size(1) for d in tqdm(dataset)] # takes a couple of minutes
    with open('len_list.obj', 'wb') as fp:
        pickle.dump(len_list, fp)
    


sampler = BucketizeBatchSampler(
    len_list,
    num_buckets=10000,
    max_token_count=30 * 16000,
    min_len=32000,
    max_len=250000,
    shuffle=True,
)

dataloader = torch.utils.data.DataLoader(
            dataset,
            batch_sampler=sampler,
        )


for X, y in dataloader:
    print(X[0][0])
    break

tensor([[0.0016, 0.0019, 0.0025,  ..., 0.0007, 0.0010, 0.0001],
        [0.0103, 0.0117, 0.0148,  ..., 0.0006, 0.0007, 0.0002],
        [0.0002, 0.0013, 0.0042,  ..., 0.0003, 0.0007, 0.0003],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


In [3]:
# model init
model = conformer_wav2vec2_pretrain_base()

In [4]:
# shapes of model output
for X, y in dataloader:
    #print(X[0].shape)
    x, lengths, mask_indices, targets, negatives, neg_idxs = model(*X)
    print('X =', X[0].shape)
    print('x =', x.shape)
    print('lengths =', lengths.shape)
    print('mask_indices =', mask_indices.shape)
    print('targets =', targets.shape)
    print('negatives =', negatives.shape)
    print('neg_idxs =', neg_idxs.shape)
    break

X = torch.Size([14, 205, 64])
x = torch.Size([14, 51, 256])
lengths = torch.Size([14])
mask_indices = torch.Size([14, 51])
targets = torch.Size([14, 9, 256])
negatives = torch.Size([100, 14, 9, 256])
neg_idxs = torch.Size([14, 900])


In [8]:
# THIS PART DOESN'T WORK 
#source https://www.internalfb.com/code/fbsource/[64489570a965cb67396c309615f727f62a9462cd]/fbcode/deeplearning/projects/pyspeech/pyspeech/criterions/wav2vec2_loss.py?lines=24%2C39

def compute_contrastive_loss(x, mask_indices, targets, neg_is_pos, reduce, logit_temp = 0.1):
    x = (
        x[mask_indices]
        .view(x.size(0), -1, x.size(-1))
        .unsqueeze(0)
        .expand(targets.shape)
    )
    logits = torch.cosine_similarity(x.float(), targets.float(), dim=-1).float()
    logits /= logit_temp
    if neg_is_pos.any():
        logits[1:][neg_is_pos] = float("-inf")
    target = logits.new_zeros(logits.size(1) * logits.size(2), dtype=torch.long)
    logits = logits.transpose(0, 2)
    logits = logits.reshape(-1, logits.size(-1))
    loss = F.cross_entropy(
        logits,
        target,
        reduction="sum" if reduce else "none",
    )
    sample_size = target.numel()
    return loss, sample_size, logits

def wav2vec_loss(net_output, reduce=True):
    
    x, lengths, mask_indices, y, negatives, neg_idxs = net_output

    # 1. get x (note that, right_context is removed in net_output if there is right_context  n src_tokens)
    #x, lengths, state, extra_output = net_output

    # 2. get negative samples
    #negatives = extra_output[f"extra_output_{self.masking_layer_key}"]

    # 3. get y and mask_indices
    #y = self.get_registered_buffer(model, "y_before_sampling")
    #mask_indices = self.get_registered_buffer(model, "mask_indices")
    assert y is not None
    assert mask_indices is not None
    if model.training:
        assert y.requires_grad
    assert mask_indices.sum() == y.shape[0] * y.shape[1]

    # 4. compute targets
    neg_is_pos = (y == negatives).all(-1)
    y = y.unsqueeze(0)
    targets = torch.cat([y, negatives], dim=0)

    # 5. compute losses
    loss, sample_size, _ = compute_contrastive_loss(
        x, mask_indices, targets, neg_is_pos, reduce
    )
    loss = loss.float()

    return loss, sample_size


epoches = 10
opt = torch.optim.SGD(model.parameters(), lr= 0.01)
for epoch in range(epoches):
    acc_loss = 0.0
    ds_size = 0
    for X, _ in tqdm(dataloader):
        ds_size += X[0].size(0)
        opt.zero_grad()
        y_h = model(*X)
        loss = wav2vec_loss(y_h)
        print(loss[1])
        loss[0].backward()
        opt.step()
        acc_loss += loss[0]
        if ds_size > 100:
            break
    print(f'ep = {epoch}, acc_loss={acc_loss}')
       

0it [00:00, ?it/s]

154


1it [2:45:24, 9924.92s/it]

154


2it [2:45:28, 4088.55s/it]

...

169


7it [00:19,  2.81s/it]

117


7it [00:22,  3.23s/it]

ep = 9, acc_loss=5420.0048828125
